In [2]:
import sys
import localSettings as ls
print(ls.main_path)

/home/david/data/searchingfornues/v08_00_00_33/cc0pinp/0217/


In [3]:
main_path = ls.main_path
sys.path.append(main_path)

In [4]:
SHRCALIBFACTOR = 1.00

In [5]:
# Doing Run3?
ISRUN3 = False

In [6]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [7]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

BNB = ""
EXT = ""
NU  = ""
NUE = ""
DRT = ""
NCPI0 = ""
CCPI0 = ""
CCNOPI = ""
NCCPI = ""
NCNOPI = ""

if ISRUN3:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_G1_beam_good_reco2_1e19'
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_G1_all_reco2'
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2'
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
    NCPI0 = 'prodgenie_nc_pi0_uboone_overlay_mcc9.1_v08_00_00_26_run3_G_reco2'
    CCPI0 = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run3_G_reco2'
    CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_run3_reco2_reco2'
    NCCPI = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_New_run3_reco2_reco2'
    NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_new_run3_reco2_reco2'
    
else:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_C1_beam_good_reco2_5e19'
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C1_all_reco2'
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2'
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
    NCPI0 = 'prodgenie_nc_pi0_uboone_overlay-v08_00_00_26_run1_reco2_reco2'
    CCPI0 = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run1_reco2'
    CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
    NCCPI = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
    NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
    
mc = uproot.open(ls.ntuple_path+NU+".root")[fold][tree]
ncpi0 = uproot.open(ls.ntuple_path+NCPI0+".root")[fold][tree]
ccpi0 = uproot.open(ls.ntuple_path+CCPI0+".root")[fold][tree]
ccnopi = uproot.open(ls.ntuple_path+CCNOPI+".root")[fold][tree]
nccpi = uproot.open(ls.ntuple_path+NCCPI+".root")[fold][tree]
ncnopi = uproot.open(ls.ntuple_path+NCNOPI+".root")[fold][tree]
nue = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]
data = uproot.open(ls.ntuple_path+BNB+".root")[fold][tree]
ext = uproot.open(ls.ntuple_path+EXT+".root")[fold][tree]
dirt = uproot.open(ls.ntuple_path+DRT+".root")[fold][tree]
lee = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]

uproot_v = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,ext,data,dirt]

variables = ["nu_e","run","sub","evt"]

nue = nue.pandas.df(variables, flatten=False)
mc = mc.pandas.df(variables, flatten=False)
ncpi0 = ncpi0.pandas.df(variables, flatten=False)
ccpi0 = ccpi0.pandas.df(variables, flatten=False)
ccnopi = ccnopi.pandas.df(variables, flatten=False)
nccpi = nccpi.pandas.df(variables, flatten=False)
ncnopi = ncnopi.pandas.df(variables, flatten=False)
data = data.pandas.df(variables, flatten=False)
ext = ext.pandas.df(variables, flatten=False)
dirt = dirt.pandas.df(variables, flatten=False)
lee = lee.pandas.df(variables, flatten=False)

In [8]:
df_v   = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,dirt,data,ext]
name_v = ['LEE','MC','NCpi0','CCpi0','CCnoPi','NCcPi','NCnoPi','nue','dirt','data','ext']

for i,df in enumerate(df_v):
    
    groups = df.groupby(['run','sub','evt','nu_e'])
    
    size = groups.size()
    n1 = float(len(np.where(size==1)[0]))
    n2 = len(np.where(size>1)[0])
    print ('in sample %s %.01f %% of entries are duplicate'%(name_v[i],100.*(n2/n1)) )

in sample LEE 0.0 % of entries are duplicate
in sample MC 0.0 % of entries are duplicate
in sample NCpi0 0.0 % of entries are duplicate
in sample CCpi0 0.0 % of entries are duplicate
in sample CCnoPi 0.0 % of entries are duplicate
in sample NCcPi 0.0 % of entries are duplicate
in sample NCnoPi 0.0 % of entries are duplicate
in sample nue 0.0 % of entries are duplicate
in sample dirt 0.0 % of entries are duplicate
in sample data 0.0 % of entries are duplicate
in sample ext 0.1 % of entries are duplicate


In [9]:
DEF_V = ["C1","C2","D2","E1","E2","F","G1","G2","G2a"]
for DEF in DEF_V:
    PI0EXT  = ls.main_path+"pi0/"+"data_extbnb_mcc9.1_v08_00_00_25_reco2_%s_pi0_reco2.root"%(DEF)
    pi0_tree = uproot.open(PI0EXT)[fold][tree]
    df = pi0_tree.pandas.df(["run", "sub","evt"])
    groups = df.groupby(['run','sub','evt'])
    size = groups.size()
    n1 = float(len(np.where(size==1)[0]))
    n2 = len(np.where(size>1)[0])
    print ('in sample EXT %s Pi0-filter %.01f %% of entries are duplicate'%(DEF,100.*(n2/n1)) )    
    
DEF_V = ["C1","E1","F","G1"]
for DEF in DEF_V:
    PI0EXT  = ls.main_path+"pi0/"+"data_bnb_mcc9.1_v08_00_00_25_reco2_%s_pi0_reco2.root"%(DEF)
    pi0_tree = uproot.open(PI0EXT)[fold][tree]
    df = pi0_tree.pandas.df(["run", "sub","evt"])
    groups = df.groupby(['run','sub','evt'])
    size = groups.size()
    n1 = float(len(np.where(size==1)[0]))
    n2 = len(np.where(size>1)[0])
    print ('in sample BNB %s Pi0-filter %.01f %% of entries are duplicate'%(DEF,100.*(n2/n1)) )

in sample EXT C1 Pi0-filter 0.0 % of entries are duplicate
in sample EXT C2 Pi0-filter 0.5 % of entries are duplicate
in sample EXT D2 Pi0-filter 0.0 % of entries are duplicate
in sample EXT E1 Pi0-filter 0.0 % of entries are duplicate
in sample EXT E2 Pi0-filter 0.0 % of entries are duplicate
in sample EXT F Pi0-filter 0.0 % of entries are duplicate
in sample EXT G1 Pi0-filter 0.2 % of entries are duplicate
in sample EXT G2 Pi0-filter 0.0 % of entries are duplicate
in sample EXT G2a Pi0-filter 0.0 % of entries are duplicate
in sample BNB C1 Pi0-filter 0.0 % of entries are duplicate
in sample BNB E1 Pi0-filter 0.7 % of entries are duplicate
in sample BNB F Pi0-filter 0.0 % of entries are duplicate
in sample BNB G1 Pi0-filter 1.2 % of entries are duplicate
